In [1]:
import json
import os
from ensure import ensure_annotations
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',None)

In [2]:
print((os.getcwd()))
os.chdir('d:\\vscode_machineLearning\\BEST_PROJECTS\\Question-Answering-system-LLMs')
print((os.getcwd()))

d:\vscode_machineLearning\BEST_PROJECTS\Question-Answering-system-LLMs\notebook
d:\vscode_machineLearning\BEST_PROJECTS\Question-Answering-system-LLMs


In [3]:
@ensure_annotations
def random_values(feature_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[feature_name].iloc[index])
            

In [4]:
@ensure_annotations
def random_values_df(col_name:str,df:pd.DataFrame,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [5]:
df = pd.read_csv(filepath_or_buffer=r'data/S08_question_answer_pairs.txt',sep='\t')

In [6]:
df = df[['Question','Answer']]

In [7]:
df.isnull().sum()

Question     19
Answer      242
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.sample(5)

,Question,Answer
776,Is Indonesia a republic?,yes
607,Is English Ghana's official language?,yes
1130,Where was Millard Fillmore born?,"in a log cabin in Summerhill, New York"
369,What does the word duck mean?,It is the common name for a number of species in the Anatidae family of birds.
291,What is one significant non-official language?,Chinese


In [10]:
## Transformers
from datasets import load_dataset , Dataset # to load dataset
from transformers import AutoTokenizer # for tokenization
from transformers import DataCollatorWithPadding # For padding
from transformers import TFAutoModelForSequenceClassification
from pathlib import Path

from transformers import DataCollatorForSeq2Seq

# seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus

In [11]:
df = Dataset.from_pandas(df)

In [12]:
train_test_df = df.train_test_split()

In [13]:
train_test_df

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 1104
    })
    test: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 369
    })
})

In [14]:
train_test_df=train_test_df.remove_columns(column_names='__index_level_0__')

In [15]:
train_test_df

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 1104
    })
    test: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 369
    })
})

## Tokenization

In [16]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

In [17]:
checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForQuestionAnswering.from_pretrained(checkpoint)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to

In [18]:
print(f"Vocab size is : {tokenizer.vocab_size}")

print(f"Model max length is : {tokenizer.model_max_length}")

print(f"Model input names are: {tokenizer.model_input_names}")

Vocab size is : 30522
Model max length is : 512
Model input names are: ['input_ids', 'attention_mask']


In [19]:
train_test_df['train']['Question']

['Have most otters fish as the primary item in their diet , supplemented by frogs , crayfish and crabs ?',
 'Is a drake a male?',
 'Who were the midnight judges?',
 'What used Singapore as a strategic trading post along the spice route?',
 'Does it have a border with Norway?',
 'Are turtle eggs leathery and soft?',
 'What expression is part of a conceptual framework for testing computer systems?',
 'What is the estimated population of Egypt?',
 'How are Isabelline penguins different from most penguins?',
 'Who died in childbirth in 1772?',
 'Does Uruguay have cold summers?',
 'Are beetles endopterygotes with complete metamorphosis?',
 'Was Abraham Lincoln the first President of the United States?',
 'Who was on the committee with Adams to draft  a Declaration of Independence?',
 'Was Coolidge the thirteenth President of the United States?',
 'Was Wilson an automobile enthusiast?',
 'Was Becquerel born in Paris into a family which, including he and his son Jean, produced four generation

In [20]:
def tokenize_dataset(examples):
    tokenized_data = tokenizer(examples['Question'],examples['Answer'], padding='longest', truncation=True)
    return {'input_ids': tokenized_data['input_ids'], 'attention_mask': tokenized_data['attention_mask']}

In [21]:
train_tokenized = train_test_df['train'].map(tokenize_dataset)
test_tokenized = train_test_df['test'].map(tokenize_dataset)

Map:   0%|          | 0/1104 [00:00<?, ? examples/s]

Map:   0%|          | 0/369 [00:00<?, ? examples/s]

In [22]:
train_tokenized['input_ids'][0]

[101,
 2031,
 2087,
 22279,
 2015,
 3869,
 2004,
 1996,
 3078,
 8875,
 1999,
 2037,
 8738,
 1010,
 20585,
 2011,
 17582,
 1010,
 13675,
 4710,
 7529,
 1998,
 26076,
 1029,
 102,
 2748,
 102]

In [23]:
test_tokenized

Dataset({
    features: ['Question', 'Answer', 'input_ids', 'attention_mask'],
    num_rows: 369
})

## Training

In [24]:
num_epochs = 3
batch_size = 32





In [25]:
train_test_df['train']['Question']

['Have most otters fish as the primary item in their diet , supplemented by frogs , crayfish and crabs ?',
 'Is a drake a male?',
 'Who were the midnight judges?',
 'What used Singapore as a strategic trading post along the spice route?',
 'Does it have a border with Norway?',
 'Are turtle eggs leathery and soft?',
 'What expression is part of a conceptual framework for testing computer systems?',
 'What is the estimated population of Egypt?',
 'How are Isabelline penguins different from most penguins?',
 'Who died in childbirth in 1772?',
 'Does Uruguay have cold summers?',
 'Are beetles endopterygotes with complete metamorphosis?',
 'Was Abraham Lincoln the first President of the United States?',
 'Who was on the committee with Adams to draft  a Declaration of Independence?',
 'Was Coolidge the thirteenth President of the United States?',
 'Was Wilson an automobile enthusiast?',
 'Was Becquerel born in Paris into a family which, including he and his son Jean, produced four generation